In [1]:
pip install requests beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install requests beautifulsoup4 python-dotenv schedule

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable


In [29]:
import os
import json
import logging
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from datetime import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import smtplib
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [43]:
!rm -f .env
!echo "EMAIL_SENDER=sendermail@gmail.com" >> .env
!echo "EMAIL_PASSWORD=11_Akshata_2" >> .env
!echo "EMAIL_RECEIVER=receiveremail@gmail.com" >> .env


'rm' is not recognized as an internal or external command,
operable program or batch file.


In [44]:
with open("env.txt", "w") as f:
    f.write("""MAIL_SENDER:sendermail@gmail.com ,
    EMAIL_PASSWORD:11_Akshata_2,
    EMAIL_RECEIVER:receiveremail@gmail.com,
    Loaded domain: UI/UX design
"""
           )

In [45]:
load_dotenv(dotenv_path="env.txt")

EMAIL_SENDER = os.getenv("EMAIL_SENDER")
EMAIL_PASSWORD = os.getenv("EMAIL_PASSWORD")
EMAIL_RECEIVER = os.getenv("EMAIL_RECEIVER")
PREFERRED_DOMAIN = os.getenv("PREFERRED_DOMAIN")
print("Sender:", EMAIL_SENDER)
print("Receiver:", EMAIL_RECEIVER)


Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 4


Sender: akshatapandit11@gmail.com
Receiver: pandithakshata@gmail.com


In [46]:
def fetch_jobs(query,limit=10):
    search_query=f"{query} site: indeed.com"
    url=f"https://www.google.com/search?q={search_query.replace(' ', '+')}"
    headers={"User-Agent": "Mozilla/5.0"}

    response=requests.get(url,headers=headers)
    soup=BeautifulSoup(response.text,"html.parser")

    jobs=[]
    seen=set()
    for link_tag in soup.find_all("a"):
        href=link_tag.get("href","")
        if "url?q" in href and "indeed.com" in href:
            link=href.split("url?q=")[1].split("&")[0]
            title=link_tag.next.strip()
            if link not in seen:
                jobs.append((title,link))
                seen.add(link)
            if len(jobs)>=limit:
                break
    return jobs

In [47]:
def fetch_jobs_from_duckduckgo(query, limit=10):
    search_query = f"{query} site:indeed.com"
    url = f"https://html.duckduckgo.com/html/?q={search_query.replace(' ', '+')}"

    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    results = []
    for a in soup.select("a.result__a")[:limit]:
        title = a.get_text(strip=True)
        href = a["href"]
        results.append((title, href))

    return results


In [48]:
def send_job_email(job_links):
    if not job_links:
        print("No job links to send.")
        return

    msg = MIMEMultipart("alternative")
    msg["Subject"] = f"🚀 {PREFERRED_DOMAIN} Jobs - Daily Update"
    msg["From"] = EMAIL_SENDER
    msg["To"] = EMAIL_RECEIVER

    # Format job links into HTML
    html = "<h3>Latest Job Listings</h3><ul>"
    for title, link in job_links:
        html += f'<li><a href="{link}" target="_blank">{title}</a></li>'
    html += "</ul>"

    part = MIMEText(html, "html")
    msg.attach(part)

    try:
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(EMAIL_SENDER, EMAIL_PASSWORD)
        server.sendmail(EMAIL_SENDER, EMAIL_RECEIVER, msg.as_string())
        server.quit()
        print(" Email sent successfully!")
    except Exception as e:
        print(" Error sending email:", e)


In [40]:
jobs = fetch_jobs(PREFERRED_DOMAIN)
send_job_email(jobs)

No job links to send.


In [42]:
jobs=fetch_jobs_from_duckduckgo(PREFERRED_DOMAIN)
send_job_email(jobs)

 Error sending email: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials d9443c01a7336-240293ab50dsm74395155ad.154 - gsmtp')
